<a href="https://colab.research.google.com/github/tcapelle/jupy2report/blob/main/Users/tcapelle/wandb/jupy2report/jupyter2report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Converting a jupyter notebook to W&B report
this is a subtitle

Let's install the required libs:

In [ ]:
# this is bash
!pip install wandb execnb ipynbname

We are going to read this notebook and create a report on W&B with it's content

In [ ]:
import wandb
import wandb.apis.reports as wb

import ipynbname
from execnb.nbio import read_nb

We can read the notebook by using `read_nb`

In [ ]:
fname = "jupyter2report.ipynb"

In [ ]:
nb = read_nb(fname)

the output is a dictionary containing the cells:

In [ ]:
nb.cells

## Creating a report

In [ ]:
api = wandb.Api()

In [ ]:
wandb.require('report-editing:v0')

In [ ]:
report = api.create_report(project='report-editing')
report.title = 'A report from a Jupyter notebook'
report.description = 'This report comes from this notebook'

let's check the content of a cell:

In [ ]:
cell = nb.cells[0]
cell

In [ ]:
nb.cells[2]

In [ ]:
nb.cells[-1]

In [ ]:
def parse_md_cell(cell):
    source = cell.source.splitlines()
    section = []
    for line in source:
        if '### ' in line:
            section += [wb.H3(line.split("### ")[1])]
        elif '## ' in line:
            section += [wb.H2(line.split("## ")[1])]
        elif '# ' in line:
            section += [wb.H1(line.split("# ")[1])]
        else:
            section += [wb.P(line)]
    return section

In [ ]:
parse_md_cell(cell)

In [ ]:
def convert_cell(cell):
    "Convert to W&B block"
    if cell.source == '':
        return []
    if cell.cell_type == "markdown":
        return parse_md_cell(cell)
    if cell.cell_type == "code":
        return [wb.CodeBlock(cell.source.splitlines())]

In [ ]:
convert_cell(cell)

In [ ]:
blocks = []
for cell in nb.cells:
    blocks += convert_cell(cell)

In [ ]:
blocks

In [ ]:
report.blocks = blocks

In [ ]:
report.save()

## Refactor

Let's put everything together, we need 1st to get the nb name automatically

In [ ]:
ipynbname.name(), ipynbname.path()

In [ ]:
def notebook2report(project, title, description, fname=None):
    "Converts this notebook into a report"
    # reads notebook
    nb = read_nb(ipynbname.path() if fname is None else fname)
    
    #create report
    api = wandb.Api()
    report = api.create_report(project=project)
    report.title = title
    report.description = description
    
    # parse cells
    blocks = []
    for cell in nb.cells:
        blocks += convert_cell(cell)
    report.blocks = blocks
    return report

In [ ]:
project='report-editing'
title = 'A report from a Jupyter notebook'
description = 'This report comes from this notebook'

report = notebook2report(project, title, description)
report.save()

### Another cool thing we could add is the cell outputs